In [1]:
import requests
import pandas as pd
import time

pd.set_option("display.max_columns", None)

In [107]:
def make_request(endpoint, params=None, record_path=None, verbose=False):
    root = "https://www.balldontlie.io/api/v1/"
    response = requests.get(root + endpoint, params=params)
    if response.status_code != 200:
        print(response.status_code)
        return response
    if verbose: 
        print("Success!")  
    df = pd.json_normalize(response.json(), record_path=record_path)
   
    # If the request ends up being a multi page request, get all the pages
    # and then complile the results into one dataframe
    n_pages = response.json()["meta"]["total_pages"] 
    if n_pages > 1:
        if isinstance(params, dict):
            for page_num in range(2, n_pages + 1):
                params.update({"page": page_num})
                response = requests.get(root + endpoint, params=params)
                page_n = pd.json_normalize(response.json(), record_path=record_path)
                df = df.append(page_n)
        if isinstance(params, list):
            for page_num in range(2, n_pages + 1):
                params.append(("page", page_num))
                response = requests.get(root + endpoint, params=params)
                page_n = pd.json_normalize(response.json(), record_path=record_path)
                df = df.append(page_n)
                params.pop()
            
    return df

In [5]:
teams = make_request("teams", record_path="data")

In [6]:
team_codes = teams[["id", "name"]]

In [7]:
team_codes

,id,name
0,1,Hawks
1,2,Celtics
2,3,Nets
3,4,Hornets
4,5,Bulls
5,6,Cavaliers
6,7,Mavericks
7,8,Nuggets
8,9,Pistons
9,10,Warriors


In [86]:
home_team_id = 30
page_num = 1

recent_games = pd.DataFrame()

res = make_request("games", record_path="data", params={"end_date": "2021-11-09",
                                                        "start_date": "2020-11-09",
                                                        "team_ids[]": [home_team_id],
                                                        "page": page_num,
                                                       "per_page": "100"})

res = res.sort_values("date", ascending=False)

res = res[res["home_team.id"].eq(home_team_id)]

recent_games = recent_games.append(res)
recent_games = recent_games.head(20)

In [2]:
from functions import make_request
from functions import get_recent_games

In [3]:
game_ids_home, game_ids_away = get_recent_games(1,2)

In [4]:
game_ids_home

[473528,
 473505,
 473454,
 473424,
 459132,
 451506,
 446517,
 445510,
 444934,
 442056,
 440182,
 438743,
 430992,
 429183,
 264794,
 264770,
 264762,
 264747,
 264710,
 264691]

In [6]:
params = "game_ids[] " * len(game_ids_home)
params = params * len(game_ids_home)
params = list(zip(params.split(" "), game_ids_home))
params.append(("per_page", 100))

In [7]:
params

[('game_ids[]', 473528),
 ('game_ids[]', 473505),
 ('game_ids[]', 473454),
 ('game_ids[]', 473424),
 ('game_ids[]', 459132),
 ('game_ids[]', 451506),
 ('game_ids[]', 446517),
 ('game_ids[]', 445510),
 ('game_ids[]', 444934),
 ('game_ids[]', 442056),
 ('game_ids[]', 440182),
 ('game_ids[]', 438743),
 ('game_ids[]', 430992),
 ('game_ids[]', 429183),
 ('game_ids[]', 264794),
 ('game_ids[]', 264770),
 ('game_ids[]', 264762),
 ('game_ids[]', 264747),
 ('game_ids[]', 264710),
 ('game_ids[]', 264691),
 ('per_page', 100)]

In [8]:
isinstance(params, list)

True

In [9]:
temp = make_request("stats", record_path="data", params=params)

In [10]:
temp

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.height_feet,player.height_inches,player.last_name,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,6128992,0,0,0,50.0,6,3,45.5,11,5,100.0,1,1,22:06,0,1,14,0,0,1,264691,2021-05-03T00:00:00.000Z,1,123,4,False,2020,Final,,25,114,17,Carmelo,NaN,NaN,Anthony,F,14,NaN,25,POR,Portland,West,Northwest,Portland Trail Blazers,Trail Blazers
1,6129016,5,0,1,0.0,1,0,25.0,8,2,0.0,0,0,14:58,1,0,4,2,0,2,264691,2021-05-03T00:00:00.000Z,1,123,4,False,2020,Final,,25,114,481,Lou,6.0,1.0,Williams,G,1,175.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
2,6129009,2,1,8,70.0,10,7,60.0,15,9,75.0,4,3,31:43,0,2,28,8,2,0,264691,2021-05-03T00:00:00.000Z,1,123,4,False,2020,Final,,25,114,167,Danilo,6.0,10.0,Gallinari,F,1,225.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
3,6128998,0,0,4,0.0,0,0,50.0,4,2,0.0,0,0,13:12,1,2,4,5,0,0,264691,2021-05-03T00:00:00.000Z,1,123,4,False,2020,Final,,25,114,253,Enes,NaN,NaN,Kanter,C,2,NaN,25,POR,Portland,West,Northwest,Portland Trail Blazers,Trail Blazers
4,6128991,8,0,2,54.5,11,6,57.9,19,11,83.3,6,5,33:35,0,0,33,2,2,3,264691,2021-05-03T00:00:00.000Z,1,123,4,False,2020,Final,,25,114,278,Damian,6.0,3.0,Lillard,G,25,195.0,25,POR,Portland,West,Northwest,Portland Trail Blazers,Trail Blazers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,7778026,0,0,0,100.0,1,1,100.0,2,2,0.0,0,0,2:07,0,0,5,0,0,1,473528,2021-11-04T00:00:00.000Z,1,98,4,False,2021,Final,,29,116,18677954,Jared,NaN,NaN,Butler,G,29,NaN,29,UTA,Utah,West,Northwest,Utah Jazz,Jazz
67,7778045,1,0,0,0.0,0,0,0.0,0,0,100.0,2,2,2:51,0,0,2,0,0,1,473528,2021-11-04T00:00:00.000Z,1,98,4,False,2021,Final,,29,116,3547289,Skylar,NaN,NaN,Mays,G,1,NaN,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
68,7778029,1,0,1,0.0,0,0,0.0,1,0,0.0,0,0,18:35,0,2,0,1,0,1,473528,2021-11-04T00:00:00.000Z,1,98,4,False,2021,Final,,29,116,3547297,Trent,NaN,NaN,Forrest,G,29,NaN,29,UTA,Utah,West,Northwest,Utah Jazz,Jazz
69,7778028,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,2:07,0,0,0,0,0,0,473528,2021-11-04T00:00:00.000Z,1,98,4,False,2021,Final,,29,116,9338307,Malik,NaN,NaN,Fitts,F,29,NaN,29,UTA,Utah,West,Northwest,Utah Jazz,Jazz


In [62]:
def get_stats(game_ids_home, game_ids_away):
    
    def format_params(game_ids):
        params = "game_ids[] " * len(game_ids_home)
        params = params * len(game_ids_home)
        params = list(zip(param_name.split(" "), game_ids_home))
        params.append(("per_page", 100))
        return params
    
    
    params_home = format_params(game_ids_home)
    stats_home = make_request("stats", record_path="data", params=params_home)
    
    params_away = format_params(game_ids_away)
    stats_away = make_request("stats", record_path="data", params=params_away)
    
    return stats_home, stats_away

In [63]:
stats_home, stats_away = get_stats(home_ids, away_ids)

In [64]:
stats_home

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.height_feet,player.height_inches,player.last_name,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,5717953,2,0,3,28.6,7,2,42.9,14,6,88.9,9,8,23:54,0,1,22,3,0,4,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,481,Lou,6.0,1.0,Williams,G,1,175.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
1,5717941,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,,0,0,0,0,0,0,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,402,Terrence,6.0,7.0,Ross,G-F,22,206.0,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
2,5717949,2,0,6,0.0,3,0,0.0,3,0,0.0,0,0,29:24,0,3,0,6,0,0,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,212,Solomon,6.0,7.0,Hill,F,1,225.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
3,5717933,0,0,0,33.3,3,1,50.0,8,4,71.4,7,5,27:00,0,2,14,0,0,1,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,196,Gary,6.0,4.0,Harris,G,22,210.0,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
4,5717944,0,2,13,0.0,0,0,58.3,12,7,0.0,0,0,26:58,6,3,14,19,0,0,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,83,Clint,6.0,10.0,Capela,C,1,240.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6058790,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0:00,0,0,0,0,0,0,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,666564,Bruno,NaN,NaN,Fernando,F,2,NaN,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
96,6058794,1,1,3,40.0,5,2,22.2,9,2,0.0,0,0,25:30,0,3,6,3,1,1,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,221,Kevin,6.0,7.0,Huerter,G,1,190.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
97,6058788,7,0,3,50.0,8,4,64.3,14,9,91.7,12,11,35:48,0,2,33,3,2,4,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,490,Trae,6.0,2.0,Young,G,1,180.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
98,6058792,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0:00,0,0,0,0,0,0,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,3091,Brandon,6.0,2.0,Goodwin,,1,180.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks


In [65]:
stats_away

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.height_feet,player.height_inches,player.last_name,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,5717953,2,0,3,28.6,7,2,42.9,14,6,88.9,9,8,23:54,0,1,22,3,0,4,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,481,Lou,6.0,1.0,Williams,G,1,175.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
1,5717941,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,,0,0,0,0,0,0,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,402,Terrence,6.0,7.0,Ross,G-F,22,206.0,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
2,5717949,2,0,6,0.0,3,0,0.0,3,0,0.0,0,0,29:24,0,3,0,6,0,0,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,212,Solomon,6.0,7.0,Hill,F,1,225.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
3,5717933,0,0,0,33.3,3,1,50.0,8,4,71.4,7,5,27:00,0,2,14,0,0,1,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,196,Gary,6.0,4.0,Harris,G,22,210.0,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
4,5717944,0,2,13,0.0,0,0,58.3,12,7,0.0,0,0,26:58,6,3,14,19,0,0,264585,2021-04-20T00:00:00.000Z,1,112,4,False,2020,Final,,22,96,83,Clint,6.0,10.0,Capela,C,1,240.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6058790,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0:00,0,0,0,0,0,0,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,666564,Bruno,NaN,NaN,Fernando,F,2,NaN,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
96,6058794,1,1,3,40.0,5,2,22.2,9,2,0.0,0,0,25:30,0,3,6,3,1,1,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,221,Kevin,6.0,7.0,Huerter,G,1,190.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
97,6058788,7,0,3,50.0,8,4,64.3,14,9,91.7,12,11,35:48,0,2,33,3,2,4,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,490,Trae,6.0,2.0,Young,G,1,180.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
98,6058792,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0:00,0,0,0,0,0,0,264675,2021-05-01T00:00:00.000Z,1,108,4,False,2020,Final,,5,97,3091,Brandon,6.0,2.0,Goodwin,,1,180.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
